In [243]:
import numpy as np
import matplotlib.pyplot as plt

In [244]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500*3)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
1500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [245]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX

In [246]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    output = [input_str[0]]

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val:
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        new_indices = [max_symbol_index]

        output.append(max_symbol)

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF, output

In [247]:
def get_word_stats(input_str: str | list, TF: list, ):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str[:-1]):
        if inp_idx <= 0:
            continue

        if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            buffer = [inp]
            prev_tf = TF[inp_idx]
            
            continue

        buffer.append(inp)
        prev_tf = TF[inp_idx]
    
    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    return WordStats

In [248]:
def show_TF(TF:list, begin = 100, length = 400):
    arr_TF = np.array(TF, dtype=float)

    arr_TF = np.pad(arr_TF, (1, 0))[:-1] - arr_TF
    arr_TF[arr_TF > 0] = 0
    arr_TF = np.abs(arr_TF)

    arr_TF = arr_TF[begin:length]

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(arr_TF.size), arr_TF)
    axs.xaxis.set_ticks(np.arange(len(input_str[begin:length])),input_str[begin:length])
    axs.xaxis.grid(True)

    plt.show()

In [249]:
def get_upstream_input(input_str: str, TF: list):
    upstream_inp: dict[int, list] = {}
    
    temp_seq = []

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str[1:-1]):
        tf = TF[inp_idx]
        
        if tf > prev_tf:
            upstream_inp.setdefault(inp_idx, temp_seq.copy())
            temp_seq.clear()

        temp_seq.append(inp)

        prev_tf = tf

    return upstream_inp

In [250]:
def get_upstream_patterns(upstream_input: list, letters: list):
    letters_dict: dict[str, int] = {}
    for let_idx, letter in enumerate(letters):
        letters_dict.setdefault(letter, let_idx)

    upstream_patterns = []

    for upstream_letters in upstream_input.values():
        upstream_pattern = np.zeros(len(letters))

        for letter in upstream_letters:
            pattern_idx = letters_dict.get(letter, 0)

            upstream_pattern[pattern_idx] = 1
        
        upstream_patterns.append(upstream_pattern)

    upstream_patterns = np.array(upstream_patterns)

    return upstream_patterns

In [251]:
def train(upstream_patterns, layer_size = 32):
    I = np.zeros(upstream_patterns.shape[1])
    O = np.zeros(layer_size)
    W = np.ones((O.size, I.size))
    Trace = np.zeros((O.size, I.size))
    print('W', W.shape, W.max(), W.min(), np.median(W))

    k = 1
    push_delta = 0.4

    prec = 1e-30

    epoch_cnt = 100
    eps0 = 0.02

    eps = 0.05
    rate_drop = 0.0

    AVG_O = np.zeros(O.shape)
    AVG_O_CNT = np.zeros(O.shape)

    samples_idx = np.arange(upstream_patterns.shape[0])

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(samples_idx)

        for i_idx in samples_idx:
            I = upstream_patterns[i_idx]

            # infer
            O = np.dot(W, I)

            # train
            winner_ids = np.argsort(O)[::-1][:1+k]
            winner = winner_ids[0]
            # loosers = winner_ids[1:1+k]

            # update
            W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
            W[winner] = np.clip(W[winner], -1, 1)

            # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
            # W[loosers] = np.clip(W[loosers], -1, 1)

            # update Trace
            rate_Trace = np.zeros(I.shape)
            rate_Trace[(I > 0.5)] = 0.3
            rate_Trace[(I < 0.5)] = -0.001

            dTrace = np.zeros(W.shape)
            dTrace[winner] = rate_Trace
            Trace += dTrace
            Trace = np.clip(Trace, 0, 1)

            eps -= eps * rate_drop
    
    return W

In [252]:
def get_next_layer_sequence(upstream_patterns, W):
    sequence = []

    for pattern in upstream_patterns:
        O = np.dot(W, pattern)

        winner = np.argmax(O)

        sequence.append(str(winner))

    return sequence

In [253]:
def downstream_train(input: list, next_input: list, upstream_inp: dict[int, list]):
    DSStats = {}

    ds_idx = 0
    ds_inp = '0'

    for inp_idx, inp in enumerate(input[1:]):
        has_ds_input = upstream_inp.get(inp_idx)
        if has_ds_input:
            ds_inp = next_input[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1
    
    return DSStats

In [254]:
max_index_L1 = 32
max_history_L1 = 1

layer_size = 32*4

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

for h_idx in range(max_history_L1):
    for wk, wv in HistoryW[h_idx].items():
        print(wk, wv)


err_L1, TF_L1, out_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)

print(err_L1, err_L1 / (len(input_str) / 100))

upstream_input = get_upstream_input(input_str=input_str, TF=TF_L1)
for item, val in upstream_input.items():
    print(item, val)
print(f'{len(upstream_input)=}')

upstream_patterns = get_upstream_patterns(upstream_input=upstream_input, letters=letters)
print(f'{upstream_patterns.shape=}')
for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

W = train(upstream_patterns, layer_size=layer_size)
print('W', W.shape, W.max(), W.min(), np.median(W))

input_str_L2 = get_next_layer_sequence(upstream_patterns, W)
print(f'{len(input_str_L2)=}')
for inp in input_str_L2:
    print(inp)

ds_stats = downstream_train(input=input_str, next_input=input_str_L2, upstream_inp=upstream_input)
print(f'{len(ds_stats)=}')
for item, val in ds_stats.items():
    print(item, val)

# WordStats_L1 = get_word_stats(input_str=input_str, TF=TF_L1)
# print(len(WordStats_L1))
# mean_WS = np.mean(np.array(list(WordStats_L1.values())))
# print(f'{mean_WS=}')
# WS_len = 0
# for key, val in WordStats_L1.items():
#     if len(key.strip()) > 1 and val >= mean_WS:
#         WS_len += 1
#     print(key, val)
# print(f'{WS_len=}')

c0 {'h': {0: 1}}
h0 {'a': {0: 1}, 't': {4: 1}}
a0 {'p': {0: 1}, 'n': {2: 1}, 't': {13: 1}}
p0 {'t': {0: 1}}
t0 {'e': {0: 1}, 'o': {31: 1}, 'l': {18: 1}}
e0 {'r': {0: 1}, 'n': {20: 1}, 's': {7: 1}, 'd': {1: 1}}
r0 {' ': {0: 1}, 'e': {11: 1}}
 0 {'i': {0: 1}, 'b': {7: 2}, 'f': {5: 1}, 'h': {31: 1}, 'w': {15: 1}, 'g': {10: 1}}
i0 {'n': {0: 2}, 's': {17: 1}}
n0 {' ': {1: 1}, 'a': {25: 1}, 'g': {5: 1}, 'd': {5: 1}}
 1 {'w': {0: 1}, 'c': {5: 2}, 'a': {18: 1}, 'n': {0: 1}, 'p': {12: 1}, 's': {9: 1}, 'i': {10: 1}, 't': {12: 1}}
w0 {'h': {1: 1}}
h1 {'i': {1: 2}, 'e': {19: 1}}
i1 {'c': {1: 1}, ' ': {12: 1}, 'n': {20: 1}, 'm': {22: 1}}
c1 {'h': {2: 1}}
h2 {' ': {2: 1}, 'r': {25: 1}}
 2 {'w': {1: 2}, 'r': {10: 2}, 'm': {9: 1}, 'i': {30: 1}, 'd': {23: 1}, 'o': {22: 1}, 's': {2: 1}}
w1 {'e': {1: 1}, 'h': {13: 1}}
e1 {' ': {3: 6}, 'n': {24: 1}}
 3 {'a': {1: 4}, 'i': {14: 2}, 's': {26: 1}, 'y': {13: 2}, 't': {19: 1}, 'f': {12: 2}, 'l': {22: 1}, 'u': {30: 1}, 'p': {20: 1}}
a1 {'r': {1: 1}, 'n': {21: 2}

In [255]:
def get_letters(W):
    letters = []

    for let_idx in range(W.shape[0]):
        letters.append(str(let_idx))

    return letters

In [256]:
max_index_L2 = 32
max_history_L2 = 4
letters_L2 = get_letters(W)

HistoryW_L2, HistoryHW_L2, IDX_L2 = predict_train(input_str_L2, 
                                                  letters_L2, 
                                                  max_history=max_history_L2, 
                                                  max_index=max_index_L2,)

for h_idx in range(max_history_L2):
    for wk, wv in HistoryW_L2[h_idx].items():
        print(wk, wv)

90 {'47': {0: 1}}
470 {'8': {0: 1}}
80 {'30': {0: 1}}
300 {'43': {0: 1}}
430 {'2': {0: 1}}
20 {'27': {0: 1}, '15': {0: 1}, '113': {2: 1}}
270 {'0': {0: 1}}
00 {'104': {0: 1}}
1040 {'2': {1: 1}}
21 {'43': {1: 1}, '108': {1: 1}, '50': {6: 1}}
431 {'36': {0: 1}}
360 {'1': {0: 1}}
10 {'114': {0: 1}}
1140 {'100': {0: 1}}
1000 {'47': {1: 1}}
471 {'2': {2: 1}}
22 {'79': {0: 1}, '71': {6: 1}, '23': {1: 1}}
790 {'43': {2: 1}}
432 {'91': {0: 1}}
910 {'102': {0: 1}}
1020 {'2': {3: 1}}
23 {'44': {0: 2}, '7': {1: 1}}
440 {'5': {0: 1}, '1': {12: 1}}
50 {'116': {0: 1}}
1160 {'67': {0: 1}}
670 {'2': {4: 1}}
24 {'52': {0: 1}, '100': {5: 1}, '23': {2: 1}}
520 {'81': {0: 1}}
810 {'116': {1: 1}}
1161 {'102': {1: 1}}
1021 {'2': {5: 1}}
25 {'47': {2: 1}, '29': {0: 1}, '75': {8: 1}}
472 {'84': {0: 1}}
840 {'81': {1: 1}}
811 {'52': {1: 1}}
521 {'99': {0: 1}}
990 {'34': {0: 1}}
340 {'28': {0: 1}}
280 {'36': {1: 1}}
361 {'2': {6: 1}}
26 {'71': {0: 1}, '12': {2: 1}, '122': {9: 1}}
710 {'97': {0: 1}}
970 {'121': 

In [257]:
err_L2, TF_L2, out_L2 = predict(input_str_L2, HistoryW_L2, max_index=max_index_L2, max_history=max_history_L2)

print(err_L2, err_L2 / (len(input_str_L2) / 100))

max_symbol='47' next_inp='47' 0 1
max_symbol='8' next_inp='8' 0 2
max_symbol='30' next_inp='30' 0 3
max_symbol='43' next_inp='43' 0 4
max_symbol='2' next_inp='2' 0 4
max_symbol='27' next_inp='27' 0 4
max_symbol='0' next_inp='0' 0 4
max_symbol='104' next_inp='104' 0 4
max_symbol='2' next_inp='2' 1 4
max_symbol='43' next_inp='43' 1 4
max_symbol='36' next_inp='36' 0 4
max_symbol='1' next_inp='1' 0 4
max_symbol='114' next_inp='114' 0 4
max_symbol='100' next_inp='100' 0 4
max_symbol='47' next_inp='47' 1 4
max_symbol='2' next_inp='2' 2 4
max_symbol='79' next_inp='79' 0 4
max_symbol='43' next_inp='43' 2 4
max_symbol='91' next_inp='91' 0 4
max_symbol='102' next_inp='102' 0 4
max_symbol='2' next_inp='2' 3 4
max_symbol='44' next_inp='44' 0 5
max_symbol='5' next_inp='5' 0 4
max_symbol='116' next_inp='116' 0 4
max_symbol='67' next_inp='67' 0 4
max_symbol='2' next_inp='2' 4 4
max_symbol='52' next_inp='52' 0 4
max_symbol='81' next_inp='81' 0 4
max_symbol='116' next_inp='116' 1 4
max_symbol='102' nex

In [258]:
def downstream_predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            upstream_inp: dict[int, list],
            DSStats: dict[str, int],
            ds_input: list,
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 0
    ds_inp = '0'

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # ds input
        has_ds_input = upstream_inp.get(inp_idx)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            ds_key = ds_inp + ':' + key
            ds_stats = DSStats.get(ds_key)
            if ds_stats:
                max_idx_val += ds_stats

            if max_idx_val > max_val and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF

err_L1_ds, TF_L1_ds = downstream_predict(input_str=input_str, 
                                        HistoryW=HistoryW, 
                                        upstream_inp=upstream_input,
                                        DSStats=ds_stats,
                                        ds_input=out_L2,
                                        max_history=max_history_L1,
                                        max_index=max_index_L1, )

print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))

max_symbol='h' next_inp='h' 11 4
max_symbol='a' next_inp='a' 0 0
max_symbol='t' next_inp='p' 15 3
max_symbol='t' next_inp='t' 0 2
max_symbol='e' next_inp='e' 0 6
max_symbol='r' next_inp='r' 0 5
max_symbol=' ' next_inp=' ' 0 8
max_symbol='i' next_inp='i' 0 5
max_symbol='s' next_inp='n' 17 5
max_symbol=' ' next_inp=' ' 1 13
max_symbol='a' next_inp='w' 18 22
max_symbol='h' next_inp='h' 1 13
max_symbol='i' next_inp='i' 1 3
max_symbol='c' next_inp='c' 1 2
max_symbol='h' next_inp='h' 2 3
max_symbol='r' next_inp=' ' 25 3
max_symbol='o' next_inp='w' 22 33
max_symbol='e' next_inp='e' 1 14
max_symbol=' ' next_inp=' ' 3 27
max_symbol='a' next_inp='a' 1 6
max_symbol=' ' next_inp='r' 20 13
max_symbol='e' next_inp='e' 2 8
max_symbol=' ' next_inp=' ' 4 16
max_symbol='i' next_inp='i' 2 5
max_symbol='n' next_inp='n' 1 3
max_symbol=' ' next_inp='t' 23 9
max_symbol=' ' next_inp='r' 31 9
max_symbol='i' next_inp='o' 3 4
max_symbol='u' next_inp='d' 8 4
max_symbol=' ' next_inp='u' 3 7
max_symbol='c' next_inp